In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBRegressor

In [ ]:
df = pd.read_csv("../input/30days-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

In [ ]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

In [ ]:
final_pred = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    encoder = OrdinalEncoder()
    xtrain[object_cols] = encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = encoder.transform(xvalid[object_cols])
    xtest[object_cols] = encoder.transform(xtest[object_cols])
    
    model = XGBRegressor(random_state=fold, n_jobs=4)
    model.fit(xtrain, ytrain)
    pred_valid = model.predict(xvalid)
    test_pred = model.predict(xtest)
    final_pred.append(test_pred)
    print(fold, mean_squared_error(yvalid, pred_valid, squared=False))

In [ ]:
pred = np.mean(np.column_stack(final_pred), axis=1)

In [ ]:
sample_submission.target = pred
sample_submission.to_csv("submission.csv", index=False)